In [1]:
from transformers import ViTConfig, ViTModel, ViTImageProcessor, ViTForImageClassification, ViTForMaskedImageModeling

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-06-11 16:43:59.080930: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 16:44:03.257673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 16:44:11.449969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
IMAGE_SIZE = 64

processor = ViTImageProcessor(size=IMAGE_SIZE)
processor

ViTImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 64,
    "width": 64
  }
}

In [3]:
from datasets import load_dataset

test_dataset = load_dataset("uoft-cs/cifar100", split="test")
test_dataset

Dataset({
    features: ['img', 'fine_label', 'coarse_label'],
    num_rows: 10000
})

In [4]:
train_dataset = load_dataset("uoft-cs/cifar100", split="train")
train_dataset

Dataset({
    features: ['img', 'fine_label', 'coarse_label'],
    num_rows: 50000
})

In [5]:
train_dataset_10 = train_dataset.train_test_split(train_size=0.1, shuffle=True, stratify_by_column="fine_label")["train"]
train_dataset_10

Dataset({
    features: ['img', 'fine_label', 'coarse_label'],
    num_rows: 5000
})

In [6]:
def process_example(example):
    inputs = processor(example['img'], return_tensors='pt')
    inputs['labels'] = example['fine_label']
    return inputs

process_example(dataset["train"][0])

NameError: name 'dataset' is not defined

In [7]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['img']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['fine_label']
    return inputs

In [13]:
prepared_ds = dataset.with_transform(transform)

In [14]:
prepared_ds["train"][0:2]

{'pixel_values': tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  0.6078,  0.4902,  0.4275],
          [ 1.0000,  1.0000,  1.0000,  ...,  0.5137,  0.4118,  0.3569],
          [ 1.0000,  1.0000,  0.9922,  ...,  0.3255,  0.2549,  0.2157],
          ...,
          [-0.1137, -0.1137, -0.1137,  ..., -0.1451,  0.0196,  0.0980],
          [-0.2471, -0.2471, -0.2471,  ..., -0.1216,  0.0196,  0.0902],
          [-0.3176, -0.3176, -0.3098,  ..., -0.1137,  0.0196,  0.0824]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  0.7020,  0.5843,  0.5216],
          [ 1.0000,  1.0000,  1.0000,  ...,  0.6000,  0.4980,  0.4431],
          [ 1.0000,  1.0000,  0.9922,  ...,  0.3961,  0.3255,  0.2863],
          ...,
          [ 0.1608,  0.1608,  0.1608,  ...,  0.0118,  0.2000,  0.2863],
          [ 0.0275,  0.0275,  0.0275,  ...,  0.0824,  0.2549,  0.3333],
          [-0.0431, -0.0431, -0.0431,  ...,  0.1216,  0.2784,  0.3569]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  0.5294,  0.3804,  0.3098],
          [ 1

In [8]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


/tmp/ipykernel_3227/311509507.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/home/jupyter/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
prepared_ds["train"].features

{'img': Image(decode=True, id=None),
 'fine_label': ClassLabel(names=['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'cra', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'tra

In [33]:
checkpoint = "/home/jupyter/datasphere/project/vit/outputs_mim_tiny_imagenet/checkpoint-47880/"

In [62]:
labels = prepared_ds['train'].features['fine_label'].names

configuration = ViTConfig.from_pretrained(checkpoint)

model = ViTForImageClassification.from_pretrained(checkpoint)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at /home/jupyter/datasphere/project/vit/outputs_mim_tiny_imagenet/checkpoint-47880/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
labels = test_dataset.features['fine_label'].names

configuration = ViTConfig(
    num_hidden_layers=3,
    num_attention_heads=16,
    hidden_size=512,
    patch_size=4,
    image_size=IMAGE_SIZE,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

model = ViTForImageClassification(configuration)

In [11]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

12815460

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-mim-pretrained-10",
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  evaluation_strategy="epoch",
  save_strategy='epoch',
  num_train_epochs=100,
  logging_steps=10,
  learning_rate=2e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  weight_decay=1e-4,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/training_args.py:1493: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=train_dataset_10.with_transform(transform),
    eval_dataset=test_dataset.with_transform(transform),
    compute_metrics=compute_metrics,
    tokenizer=processor
)


In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

  0%|          | 10/4000 [00:13<57:44,  1.15it/s] 

{'loss': 4.6049, 'grad_norm': 5.1431379318237305, 'learning_rate': 1.9950000000000004e-05, 'epoch': 0.25}


  0%|          | 20/4000 [00:21<53:47,  1.23it/s]

{'loss': 4.4323, 'grad_norm': 5.134486675262451, 'learning_rate': 1.9900000000000003e-05, 'epoch': 0.5}


  1%|          | 30/4000 [00:29<53:34,  1.23it/s]

...

{'loss': 1.1684, 'grad_norm': 108.92047882080078, 'learning_rate': 8.6e-06, 'epoch': 57.0}



 97%|█████████▋| 77/79 [00:26<00:00,  2.83it/s]
                                                     
100%|██████████| 79/79 [00:27<00:00,  2.89it/s]
                                               

{'eval_loss': 3.4715256690979004, 'eval_accuracy': 0.2076, 'eval_runtime': 27.5841, 'eval_samples_per_second': 362.528, 'eval_steps_per_second': 2.864, 'epoch': 57.0}


 57%|█████▋    | 2288/4000 [1:01:03<45:51,  1.61s/it]  

KeyboardInterrupt: 